In [8]:
import os
import re
import subprocess
import numpy as np

In [9]:
tags = [t.strip().split(' ')[-1] for t in list(filter(lambda x:x != '-DOCSTART- -X- -X- -X- O\n',open("../data/conll03/deu.train", encoding="ISO-8859-1").readlines())) if len(t.strip().split(' '))>1]
tags = sorted(list(set(tags)))
tags

['B-LOC', 'B-MISC', 'B-ORG', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']

In [10]:
def if_has_W(feature):
    if feature not in W:
        return 0
    else:
        return W[feature]


def Pre_Next(): # ('NN','NN')
    P_tags  =[] 
    for pt in tags:
        tp_P_tags = []
        for nt in tags:
            tp_P_tags.append(if_has_W(('pt '+nt,'t '+pt)))
        P_tags.append(tp_P_tags)
    P_tags = np.asarray(P_tags)
    return P_tags


def hmm_viterbi(W, words):
    y_pred = []
    i= 0
    aug_words = ['__SENTINEL__ O', '__SENTINEL__ O'] + words + ['__SENTINEL__ O', '__SENTINEL__ O']
    for ppw_, pw_, w_, nw_, nnw_ in zip(aug_words, aug_words[1:], aug_words[2:], aug_words[3:], aug_words[4:]):
        # NOTE: No pt, ppt attribs currently
        ppw = ppw_.split(' ')[0]
        pw = pw_.split(' ')[0]
        w = w_.split(' ')[0]
        nw = nw_.split(' ')[0]
        nnw = nnw_.split(' ')[0]

        ppt = ppw_.split(' ')[1]
        pt = pw_.split(' ')[1]
        t = w_.split(' ')[1]
        nt = nw_.split(' ')[1]
        nnt = nnw_.split(' ')[1]


        attribs = {'ppw ' + ppw,
                   'pw ' + pw,
                   'w ' + w,
                   'nw ' + nw,
                   'nnw ' + nnw,
                   'ppt '+ppt,
                   'pt '+pt,
                   't '+t,
                   'nt '+nt,
                   'nnt '+nnt
                   }

        if re.search('[0-9]', w):
            attribs.add('dig __NONE__')
        if re.search('[A-Z]', w):
            attribs.add('uc __NONE__')
        if '-' in w:
            attribs.add('hyph __NONE__')
        
        score_list = []
        for tag in tags:
            score = 0
            for a in attribs:
                if (a, tag) in W:
                    score += W[(a, tag)]
            score_list.append(score)
        
        #Viterbi
        if i == 0:
            data = words[i]
            first_level = []
            for tag_number in range(len(tags)):   
                first_level.append((if_has_W(('pt '+'','t '+tag))+score_list[tag_number]))
            y_pred.append(tags[first_level.index(max(first_level))])
            next_value = first_level
        else:
            data = words[i]
            probility = ((next_value + Pre_Next()).T + [score_list[tag_number] for tag_number in range(len(tags))]).T
            y_pred.append(tags[probility.max(axis=1).argmax()])
            next_value = probility.max(axis=1)

        i+=1
    return y_pred

def create_features(words, word_tags):
    loc = 0
    w_temp = {}
    aug_words = ['__SENTINEL__ O', '__SENTINEL__ O'] + words + ['__SENTINEL__ O', '__SENTINEL__ O']

    for ppw_, pw_, w_, nw_, nnw_ in zip(aug_words, aug_words[1:], aug_words[2:], aug_words[3:], aug_words[4:]):
        # NOTE: No pt, ppt attribs currently
        ppw = ppw_.split(' ')[0]
        pw = pw_.split(' ')[0]
        w = w_.split(' ')[0]
        nw = nw_.split(' ')[0]
        nnw = nnw_.split(' ')[0]

        ppt = ppw_.split(' ')[1]
        pt = pw_.split(' ')[1]
        t = w_.split(' ')[1]
        nt = nw_.split(' ')[1]
        nnt = nnw_.split(' ')[1]


        attribs = {'ppw ' + ppw,
                   'pw ' + pw,
                   'w ' + w,
                   'nw ' + nw,
                   'nnw ' + nnw
                   }

        if re.search('[0-9]', w):
            attribs.add('dig __NONE__')
        if re.search('[A-Z]', w):
            attribs.add('uc __NONE__')
        if '-' in w:
            attribs.add('hyph __NONE__')

        for a in attribs:
            if (a, word_tags[loc]) in w_temp:
                w_temp[(a, word_tags[loc])] += 1
            else: w_temp[(a, word_tags[loc])] = 1

        attribs = {
                   'ppt '+ppt,
                   'pt '+pt,
                   't '+t,
                   'nt '+nt,
                   'nnt '+nnt
                    }

        for a in attribs:
            if (a, word_tags[loc]) in w_temp:
                w_temp[(a, word_tags[loc])] += 0.01
            else: w_temp[(a, word_tags[loc])] = 0.01
        loc += 1
        
        
    for i in range(len(word_tags)+1):
        if i == 0:
            first_tag = 'pt '+''
        else:
            first_tag = 'pt '+word_tags[i-1]
        
        if i == len(word_tags):
            next_tag = 't '+''
        else:
            next_tag = 't '+word_tags[i]
            
            if (first_tag,next_tag) not in w_temp:
                w_temp[(first_tag,next_tag)] = 0.001#create_trans((tags[i-1],tags[i]))
            else:
                w_temp[(first_tag,next_tag)] += 0.001#create_trans((tags[i-1],tags[i]))

    return w_temp

In [11]:
text = [t.strip().split(' ') for t in list(filter(lambda x:x != '-DOCSTART- -X- -X- -X- O\n',open("../data/conll03/deu.train", encoding="ISO-8859-1").readlines()))]
sen_locs = [t for t in range(len(text)) if len(text[t])==1]
text_train = [[text[s] for s in range(sen_locs[loc]+1,sen_locs[loc+1])] for loc in range(len(sen_locs)-1)]
text_train[:1]

[[['Ereignis', 'Ereignis', 'NN', 'I-NC', 'O'],
  ['und', 'und', 'KON', 'O', 'O'],
  ['Erzählung', 'Erzählung', 'NN', 'I-NC', 'O'],
  ['oder', 'oder', 'KON', 'I-NC', 'O'],
  [':', ':', '$.', 'O', 'O']]]

In [12]:
text = [t.strip().split(' ') for t in list(filter(lambda x:x != '-DOCSTART- -X- -X- -X- O\n',open("../data/conll03/deu.testa", encoding="ISO-8859-1").readlines()))]
sen_locs = [t for t in range(len(text)) if len(text[t])==1]
text_pred = [[text[s] for s in range(sen_locs[loc]+1,sen_locs[loc+1])] for loc in range(len(sen_locs)-1)]
text_pred[:1]

[[['Großer', 'Große', 'NN', 'I-NC', 'O'],
  ['Foto-Wettbeweb', '<unknown>', 'NN', 'I-NC', 'O'],
  ['"', '"', '$(', 'O', 'O'],
  ['Nordendler', '<unknown>', 'NN', 'I-NC', 'I-ORG'],
  ['"', '"', '$(', 'O', 'O'],
  ['laden', 'laden', 'VVFIN', 'I-VC', 'O'],
  ['die', 'd', 'ART', 'I-NC', 'O'],
  ['Nordendler', '<unknown>', 'NN', 'I-NC', 'I-MISC'],
  ['ein', 'ein', 'ART', 'B-NC', 'O']]]

In [13]:
W = {}
epoch = 100
for e in range(epoch):
    print('epoch = '+ str(e+1))
    # training
    for sentence in text_train:

        words = [' '.join([s[0],s[1]]) for s in sentence]
        word_tags = [s[-1] for s in sentence]

        y_pred = hmm_viterbi(W, words)
        w_true = create_features(words, word_tags)
        w_pred = create_features(words, y_pred)
        for wt in w_true:
            if wt in W: W[wt] += w_true[wt]
            else: W[wt] = w_true[wt]

        for wp in w_pred:
            if wp in W: W[wp] -= w_pred[wp]
            else: W[wp] = -w_pred[wp]
                
    # predicting            
    filename = 'SPViterbi_5attribs_pos_viterbi_deu.txt'
    if os.path.exists(filename):
        os.remove(filename)

    save_wt = '\n'
    for sentence in text_pred:
        words = [' '.join([s[0],s[1]]) for s in sentence]
        y_pred = hmm_viterbi(W, words)

        for wt in range(len(sentence)):
            save_wt += sentence[wt][0]+' '+sentence[wt][1]+' '+sentence[wt][2]+' '+sentence[wt][3]+' '+sentence[wt][4]+' '+y_pred[wt]+'\n'

        save_wt += '\n'

    with open('SPViterbi_5attribs_pos_viterbi_deu.txt', 'w+') as txt_file:
        txt_file.write(save_wt)

    output = subprocess.run("/usr/bin/perl -w conlleval < SPViterbi_5attribs_pos_viterbi_deu.txt", shell=True, stdout=subprocess.PIPE, universal_newlines=True)
    print('\n'+output.stdout)

epoch = 1

processed 51444 tokens with 4833 phrases; found: 2814 phrases; correct: 1191.
accuracy:  90.07%; precision:  42.32%; recall:  24.64%; FB1:  31.15
              LOC: precision:  40.86%; recall:  24.22%; FB1:  30.41
             MISC: precision:  50.50%; recall:  15.05%; FB1:  23.19
              ORG: precision:  41.76%; recall:  20.23%; FB1:  27.25
              PER: precision:  41.42%; recall:  35.83%; FB1:  38.42

epoch = 2

processed 51444 tokens with 4833 phrases; found: 3199 phrases; correct: 1486.
accuracy:  90.65%; precision:  46.45%; recall:  30.75%; FB1:  37.00
              LOC: precision:  53.05%; recall:  41.24%; FB1:  46.40
             MISC: precision:  53.33%; recall:  13.47%; FB1:  21.50
              ORG: precision:  37.98%; recall:  23.29%; FB1:  28.87
              PER: precision:  45.38%; recall:  40.97%; FB1:  43.06

epoch = 3

processed 51444 tokens with 4833 phrases; found: 3667 phrases; correct: 1600.
accuracy:  91.07%; precision:  43.63%; recall:  33.

KeyboardInterrupt: 